# 각 클래스 이미지 확인

In [ ]:
import numpy as np
import pandas as pd

import os
import cv2
import glob
import json

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import Image

import warnings
warnings.filterwarnings(action='ignore')

from datetime import datetime
from datetime import time

## Loading training meta data (json)

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/building_train.csv')

In [ ]:
df_train.head(5)

,Unnamed: 0,image_id,type_id,type_name,geo_coord,geo_type,object_imcoords,building_imcoords,road_imcoords,ingest_time
0,0,BLD00001_PS3_K3A_NIA0276.png,2,아파트,"[[31.4347031225, 30.0413951468, 0.0], [31.4405...",Polygon,EMPTY,"856.2952642894032,1000.8046984782792,828.00831...",EMPTY,2020-10-27T02:04:23.355595Z
1,1,BLD00001_PS3_K3A_NIA0276.png,2,아파트,"[[31.4347031225, 30.0413951468, 0.0], [31.4405...",Polygon,EMPTY,"821.5515067078223,969.1356097092277,799.413891...",EMPTY,2020-10-27T02:04:23.355595Z
2,2,BLD00001_PS3_K3A_NIA0276.png,2,아파트,"[[31.4347031225, 30.0413951468, 0.0], [31.4405...",Polygon,EMPTY,"761.5954648635015,999.8822978345204,747.451988...",EMPTY,2020-10-27T02:04:23.355595Z
3,3,BLD00001_PS3_K3A_NIA0276.png,2,아파트,"[[31.4347031225, 30.0413951468, 0.0], [31.4405...",Polygon,EMPTY,"791.4197523450355,949.7651961902932,767.129868...",EMPTY,2020-10-27T02:04:23.355595Z
4,4,BLD00001_PS3_K3A_NIA0276.png,2,아파트,"[[31.4347031225, 30.0413951468, 0.0], [31.4405...",Polygon,EMPTY,"730.8487767382088,982.3566856031034,707.481293...",EMPTY,2020-10-27T02:04:23.355595Z


In [ ]:
# class 개수 확인
df_train['type_name'].unique()

array(['아파트', '소형 시설', '중형 단독 시설', '공장', '기타', '대형 시설', '컨테이너 박스', nan],
      dtype=object)

In [ ]:
# null 값 확인
df_train.isnull().sum()

Unnamed: 0            0
image_id              0
type_id               0
type_name             1
geo_coord             0
geo_type              0
object_imcoords       0
building_imcoords    10
road_imcoords         0
ingest_time           0
dtype: int64

In [ ]:
# 삭제
df_train = df_train.dropna(axis=0)

In [ ]:
# null 값 확인
df_train.isnull().sum()

Unnamed: 0           0
image_id             0
type_id              0
type_name            0
geo_coord            0
geo_type             0
object_imcoords      0
building_imcoords    0
road_imcoords        0
ingest_time          0
dtype: int64

### 불필요 컬럼 제거

In [ ]:
df_train = df_train[['image_id','type_id', 'geo_coord', 'type_name','building_imcoords','ingest_time']]
df_train = df_train.sort_values(by = ['image_id','type_id']).reset_index(drop=True)
df_train.head(10)

,image_id,type_id,geo_coord,type_name,building_imcoords,ingest_time
0,BLD00001_PS3_K3A_NIA0276.png,1,"[[31.4347031225, 30.0413951468, 0.0], [31.4405...",소형 시설,"532.6456985368046,314.92411965791746,549.44221...",2020-10-27T02:04:23.355595Z
1,BLD00001_PS3_K3A_NIA0276.png,2,"[[31.4347031225, 30.0413951468, 0.0], [31.4405...",아파트,"856.2952642894032,1000.8046984782792,828.00831...",2020-10-27T02:04:23.355595Z
2,BLD00001_PS3_K3A_NIA0276.png,2,"[[31.4347031225, 30.0413951468, 0.0], [31.4405...",아파트,"821.5515067078223,969.1356097092277,799.413891...",2020-10-27T02:04:23.355595Z
3,BLD00001_PS3_K3A_NIA0276.png,2,"[[31.4347031225, 30.0413951468, 0.0], [31.4405...",아파트,"761.5954648635015,999.8822978345204,747.451988...",2020-10-27T02:04:23.355595Z
4,BLD00001_PS3_K3A_NIA0276.png,2,"[[31.4347031225, 30.0413951468, 0.0], [31.4405...",아파트,"791.4197523450355,949.7651961902932,767.129868...",2020-10-27T02:04:23.355595Z
5,BLD00001_PS3_K3A_NIA0276.png,2,"[[31.4347031225, 30.0413951468, 0.0], [31.4405...",아파트,"730.8487767382088,982.3566856031034,707.481293...",2020-10-27T02:04:23.355595Z
6,BLD00001_PS3_K3A_NIA0276.png,2,"[[31.4347031225, 30.0413951468, 0.0], [31.4405...",아파트,"760.3655973384898,929.7798489088528,736.383180...",2020-10-27T02:04:23.355595Z
7,BLD00001_PS3_K3A_NIA0276.png,2,"[[31.4347031225, 30.0413951468, 0.0], [31.4405...",아파트,"698.2572873253985,966.9833415404572,680.731675...",2020-10-27T02:04:23.355595Z
8,BLD00001_PS3_K3A_NIA0276.png,2,"[[31.4347031225, 30.0413951468, 0.0], [31.4405...",아파트,"726.8517072819207,910.4094353899184,706.558893...",2020-10-27T02:04:23.355595Z
9,BLD00001_PS3_K3A_NIA0276.png,2,"[[31.4347031225, 30.0413951468, 0.0], [31.4405...",아파트,"715.7828995568153,910.1019685086655,707.481293...",2020-10-27T02:04:23.355595Z


In [ ]:
df_etc = df_train[df_train.type_name=='기타']
df_etc

,image_id,type_id,geo_coord,type_name,building_imcoords,ingest_time
542,BLD00004_PS3_K3A_NIA0276.png,7,"[[31.4114256639, 30.0360343801, 0.0], [31.4172...",기타,"636.5483868889218,635.0638221099634,637.703048...",2020-11-19T02:38:43.713621Z
543,BLD00004_PS3_K3A_NIA0276.png,7,"[[31.4114256639, 30.0360343801, 0.0], [31.4172...",기타,"415.86740406859434,914.3844531725146,416.30393...",2020-11-19T02:38:43.713621Z
9081,BLD00228_PS3_K3A_NIA0276.png,7,"[[31.4414170765, 29.9855761431, 0.0], [31.4472...",기타,"13.305466166827014,708.8699421645712,13.871656...",2020-11-12T02:40:17.185040Z
9082,BLD00228_PS3_K3A_NIA0276.png,7,"[[31.4414170765, 29.9855761431, 0.0], [31.4472...",기타,"79.26660695130987,709.4361322142233,79.5497019...",2020-11-12T02:40:17.185040Z
30308,BLD01015_PS3_K3A_NIA0277.png,7,"[[31.3565313155, 30.0539311821, 0.0], [31.3623...",기타,"102.23161851068828,629.3569212574827,105.75684...",2020-09-09T01:47:33.212599Z
30309,BLD01015_PS3_K3A_NIA0277.png,7,"[[31.3565313155, 30.0539311821, 0.0], [31.3623...",기타,"195.29512467994684,618.3704882828632,194.14180...",2020-09-09T01:47:33.212599Z
30310,BLD01015_PS3_K3A_NIA0277.png,7,"[[31.3565313155, 30.0539311821, 0.0], [31.3623...",기타,"237.198999857337,614.1416568429431,235.0845841...",2020-09-09T01:47:33.212599Z
33718,BLD01083_PS3_K3A_NIA0277.png,7,"[[31.3100693411, 30.0380976402, 0.0], [31.3159...",기타,"838.7682362577759,852.7477068620722,870.883236...",2020-11-18T05:32:18.446315Z
33719,BLD01083_PS3_K3A_NIA0277.png,7,"[[31.3100693411, 30.0380976402, 0.0], [31.3159...",기타,"704.9473123129279,935.2412973061626,790.389109...",2020-11-18T05:32:18.446315Z
33720,BLD01083_PS3_K3A_NIA0277.png,7,"[[31.3100693411, 30.0380976402, 0.0], [31.3159...",기타,"461.2389409079544,726.6273070169768,468.465155...",2020-11-18T05:32:18.446315Z


In [ ]:
df_image = df_etc.image_id.value_counts().to_frame(name = 'bld_count') # 빌딩 카운터 컬럼 넣기
df_image = df_image.join(df_etc[['type_name','image_id']].drop_duplicates()['image_id'].value_counts()).rename(columns = {'image_id':'bld_nunique'})
df_img2bld = df_image.join(pd.crosstab(df_etc['image_id'], df_etc['type_name'])[df_etc['type_name'].value_counts().index])
df_img2bld

,bld_count,bld_nunique,기타
BLD01015_PS3_K3A_NIA0277.png,3,1,3
BLD01083_PS3_K3A_NIA0277.png,3,1,3
BLD00004_PS3_K3A_NIA0276.png,2,1,2
BLD00228_PS3_K3A_NIA0276.png,2,1,2
BLD04065_PS3_K3A_NIA0377.png,2,1,2
BLD09468_PS3_K3A_NIA0386.png,1,1,1
OBJ02622_PS3_K3_NIA0126.png,1,1,1
OBJ03847_PS3_K3A_NIA0161.png,1,1,1


In [ ]:
Image('/content/drive/MyDrive/SharedData/sia-satellite-data/Training/train_buildings_data/BLD01015_PS3_K3A_NIA0277.png')

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
fig = plt.figure(figsize = (40,40))
rows = 8
cols = 1

img1 = cv2.imread('/content/drive/MyDrive/SharedData/sia-satellite-data/Training/train_buildings_data/BLD01015_PS3_K3A_NIA0277.png')
img2 = cv2.imread('/content/drive/MyDrive/SharedData/sia-satellite-data/Training/train_buildings_data/BLD01083_PS3_K3A_NIA0277.png')
img3 = cv2.imread('/content/drive/MyDrive/SharedData/sia-satellite-data/Training/train_buildings_data/BLD00004_PS3_K3A_NIA0276.png')
img4 = cv2.imread('/content/drive/MyDrive/SharedData/sia-satellite-data/Training/train_buildings_data/BLD00228_PS3_K3A_NIA0276.png')
img5 = cv2.imread('/content/drive/MyDrive/SharedData/sia-satellite-data/Training/train_buildings_data/BLD04065_PS3_K3A_NIA0377.png')
img6 = cv2.imread('/content/drive/MyDrive/SharedData/sia-satellite-data/Training/train_buildings_data/BLD09468_PS3_K3A_NIA0386.png')
img7 = cv2.imread('/content/drive/MyDrive/SharedData/sia-satellite-data/Training/train_buildings_data/OBJ02622_PS3_K3_NIA0126.png')
img8 = cv2.imread('/content/drive/MyDrive/SharedData/sia-satellite-data/Training/train_buildings_data/OBJ03847_PS3_K3A_NIA0161.png')
 

ax1 = fig.add_subplot(rows, cols, 1)
ax1.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
ax1.axis("off")
 
ax2 = fig.add_subplot(rows, cols, 2)
ax2.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
ax2.axis("off")

ax3 = fig.add_subplot(rows, cols, 3)
ax3.imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))
ax3.axis("off")
 
ax4 = fig.add_subplot(rows, cols, 4)
ax4.imshow(cv2.cvtColor(img4, cv2.COLOR_BGR2RGB))
ax4.axis("off")

ax5 = fig.add_subplot(rows, cols, 5)
ax5.imshow(cv2.cvtColor(img5, cv2.COLOR_BGR2RGB))
ax5.axis("off")
 
ax6 = fig.add_subplot(rows, cols, 6)
ax6.imshow(cv2.cvtColor(img6, cv2.COLOR_BGR2RGB))
ax6.axis("off")

ax7 = fig.add_subplot(rows, cols, 7)
ax7.imshow(cv2.cvtColor(img7, cv2.COLOR_BGR2RGB))
ax7.axis("off")
 
ax8 = fig.add_subplot(rows, cols, 8)
ax8.imshow(cv2.cvtColor(img8, cv2.COLOR_BGR2RGB))
ax8.axis("off")

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.